In [1]:
import mea_analysis_pipeline as msp
import helper_functions as helper

In [ ]:
import importlib
importlib.reload(msp)
importlib.reload(helper)

In [ ]:
file_path = '/mnt/disk15tb/mmpatil/Spikesorting/Data/fullscan_5min_16757.raw.h5'


In [ ]:

rec_numbers = [ 0 ] # need [0],locations[8][1]to know it while recording

time_in_s = 300
for rec_number in rec_numbers:
    msp.process_block(rec_number,file_path,time_in_s)

In [ ]:
sorter_path = '/mnt/disk15tb/mmpatil/Spikesorting/sorter_output/sorter_5min_16757/'

In [ ]:
recording,rec_name = msp.get_data_maxwell(file_path,9)
print(rec_name)

In [ ]:
sorting = msp.get_kilosort_result(sorter_path+'block_rec0009/kilosort3_rec0009/sorter_output')
waveforms = msp.get_waveforms_result(sorter_path+'block_rec0009/waveforms_rec0009',sorter=sorting)
waveforms_good = msp.get_waveforms_result(sorter_path+'block_rec0009/waveforms_good_rec0009',sorter=sorting)

In [ ]:
sorting = msp.get_kilosort_result(f"{sorter_path}block_{rec_name}/kilosort3_{rec_name}/sorter_output")
waveforms = msp.get_waveforms_result(f"{sorter_path}block_{rec_name}/waveforms_{rec_name}",sorter=sorting)
waveforms_good = msp.get_waveforms_result(f"{sorter_path}block_{rec_name}/waveforms_good_{rec_name}",sorter=sorting)

In [ ]:

msp.analyse_waveforms_sigui(waveforms_good) #Need to implement compute noise levels.

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 500)
metrics = msp.get_quality_metrics(waveforms_good)
metrics

In [ ]:
indexes = metrics[(metrics['firing_rate']<0.5 )|(metrics['isi_violations_ratio']>=1)].index

In [ ]:
print(indexes)

In [ ]:
import spikeinterface.widgets as sw
import spikeinterface.full
%matplotlib widget
sw.plot_probe_map(recording,with_channel_ids=False)

In [ ]:
job_kwargs = dict(n_jobs=64, chunk_duration="1s", progress_bar=True)
waveforms_good.run_extract_waveforms(**job_kwargs)

In [ ]:
import spikeinterface.postprocessing as sp
import matplotlib.pyplot as plt
import spikeinterface.widgets as sw
%matplotlib widget
locations = sp.compute_unit_locations(waveforms_good)
print(type(locations))
import numpy as np
#np.savetxt("unitloc_10mins.txt",locations)
ax = plt.subplot(111)
sw.plot_probe_map(recording,ax=ax,with_channel_ids=False)
for x,y in locations:
    ax.scatter(x,y)
plt.show()

In [ ]:
import spikeinterface.full
unit_extremum_channel =spikeinterface.full.get_template_extremum_channel(waveforms_good, peak_sign='neg')
print(unit_extremum_channel)

x = [int(x) for x in unit_extremum_channel.values()]
x.sort()
print(x)

In [ ]:
import numpy as np
channel_ids = spikeinterface.full.get_closest_channels(recording,num_channels=9)
print(channel_ids[0])
indices = np.where(channel_ids[0] ==571)
row_index = indices[0][0]

# Retrieve the row
row = channel_ids[0][row_index]


row = [ str(x) for x in row]

print(row)

In [ ]:
sw.plot_timeseries(recording,channel_ids=row,time_range=[0.5,0.6],return_scaled=True,show_channel_ids=True)

In [ ]:
row = [ '8','31','869','895','214','423','701','923','118']
list_traces = []

traces = recording.get_traces(

        channel_ids=row,
        start_frame=int(0.825 *20000),
        end_frame=int(0.925*20000),

    )
print(traces)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scalebury
%matplotlib widget
# Generate sample data


## Set up the figure and axis
fig, ax = plt.subplots(nrows=9, ncols=1, sharex=True)

# Loop through each column and plot it on a separate axis
for i in range(9):
    ax[i].plot(traces[:, i])
    ax[i]=scalebury.add_scalebar(ax[i])
    #ax[i].set_ylabel(f"C{row[i]}")
    #ax[i].set_ylim([480, 540])
# Set the x-axis label on the bottom subplot
#ax[-1].set_xlabel("samples ( Sampling rate 20K)")

# Show the plot
plt.show()

In [ ]:
row = [ '8','31','869','895','214','423','701','923','118']
list_traces = []

traces = recording.get_traces(

        channel_ids=row,
        start_frame=int(0.869 *20000),
        end_frame=int(0.872*20000),

    )
print(traces)

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import scalebury
# Generate sample data


## Set up the figure and axis
fig, ax = plt.subplots(nrows=9, ncols=1, sharex=True)

# Loop through each column and plot it on a separate axis
for i in range(9):
    ax[i].plot(traces[:, i])
    ax[i]=scalebury.add_scalebar(ax[i])
    #ax[i].set_ylabel(f"C{row[i]}")
    #ax[i].set_ylim([480, 540])
# Set the x-axis label on the bottom subplot
#ax[-1].set_xlabel("samples ( Sampling rate 20K)")

# Show the plot
plt.show()

In [ ]:
import scalebury
#obj = scalebury.AnchoredScaleBar(ax.transpose)
ax2 = scalebury.add_scalebar(ax[1])
plt.show()

In [ ]:
sw.plot_unit_templates(waveforms_good,channel_ids = row)

In [ ]:
sw.plot_unit_waveforms(waveforms_good,unit_ids=[112])

In [ ]:
channel_ids = recording.get_channel_ids()
print(channel_ids)
channel_association_dict = {int(y):x for x,y in enumerate(channel_ids) }
print(channel_association_dict)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
for i, unit_id in enumerate([112]):
    template = waveforms.get_waveforms(unit_id)
    
    ax.plot(template.T, lw=3, color ='Fuchsia')
print(template.shape)

In [ ]:
data = traces
data_scaled = (data - np.mean(data, axis=0)) / np.std(data, axis=0)
fig = plt.figure()
for i, channel in enumerate(row):
    plt.plot(data_scaled[:, i],label = channel)

# Show the plot
plt.legend()
plt.show()

In [ ]:


sw.plot_unit_waveforms(waveforms_good,unit_ids = [112])

In [ ]:
sw.plot_unit_templates(waveforms_good,unit_ids = [112])

In [ ]:
msp.export_to_phy_datatype(waveforms_good)

In [ ]:
!phy extract-waveforms /home/mmpatil/Documents/spikesorting/MEA_Analysis/Python/phy_folder/params.py

In [ ]:
!phy template-gui  /home/mmpatil/Documents/spikesorting/MEA_Analysis/Python/phy_folder/params.py

In [ ]:
non_violated_units = msp.remove_violated_units(sorting1min,waveforms1min)
print(non_violated_units)
template_channel_dict = msp.get_unique_templates_channels(non_violated_units,waveforms1min)
non_redundant_templates = list(template_channel_dict.keys())
print(non_redundant_templates)

In [ ]:

import os
os.chdir('/home/mmpatil/Documents/spikesorting/MEA_Analysis/Python/sorter_workspace/block_rec0005')
waveform_good = waveforms1min.select_units(non_redundant_templates,new_folder='waveforms_good_rec0005')
waveform_good.run_extract_waveforms()

In [ ]:
print(waveforms_good.unit_ids)

In [ ]:
template1 = waveforms_good.get_template(unit_id = 8)
template1 = template1.transpose()
print(template1.shape)

In [ ]:
locations = waveforms_good.get_channel_locations()



In [ ]:
waveforms_good.run_extract_waveforms()

In [ ]:
%matplotlib widget
import spikeinterface.widgets as sw
import spikeinterface.postprocessing as sp
import matplotlib.pyplot as plt
locations = sp.compute_unit_locations(waveforms_good)
print(locations.shape)
#ax = plt.subplot(111)
sw.plot_unit_waveforms(waveforms_good, unit_ids=[8])
print(locations)

In [ ]:
good_units = msp.remove_violated_units(metrics)
print(good_units)

In [ ]:
recording,rec_name = msp.get_data_maxwell(file_path,6)
channel_location_dict = msp.get_channel_locations_mapping(recording)
print(channel_location_dict)

In [ ]:
import spikeinterface.full
unit_extremum_channel =spikeinterface.full.get_template_extremum_channel(waveforms1min, peak_sign='neg')
    #Step 1: keep only units that are in good_units 
unit_extremum_channel = {key:value for key,value in unit_extremum_channel.items() if key in good_units}

In [ ]:
# New dictionary with combined information
new_dict = {}
for template, channel in unit_extremum_channel.items():
    if template not in new_dict:
        new_dict[template] = {}

# Add an entry for this template and its corresponding location to the new dictionary
    new_dict[template][channel] = [int(channel_location_dict[channel][0]/17.5),int(channel_location_dict[channel][1]/17.5)]
print(new_dict)

In [ ]:
import axon_velocity as av

gtr = av.compute_graph_propagation_velocity(template=template1, locations=locations, fs=20000)

In [ ]:
templates = helper.get_templates_with_same_channels('/home/mmpatil/Documents/spikesorting/MEA_Analysis/Python/Electrodes_rec0010')
print(templates)

In [ ]:
metrics = msp.get_quality_metrics(waveforms_good)

In [ ]:
metrics

In [ ]:
%matplotlib widget

In [ ]:
import spikeinterface.postprocessing as sp

locations = sp.compute_unit_locations(waveforms1min)
print(locations)
import numpy as np
#np.savetxt("unitloc_10mins.txt",locations)
ax = plt.subplot(111)
sw.plot_probe_map(recording,ax=ax,with_channel_ids=False)
for x,y in locations:
    ax.scatter(x,y)


In [ ]:
channel_locations = recording_chunk.get_channel_locations()
channel_ids = recording_chunk.get_channel_ids()
channel_locations_mappings= {channel_id: location for location, channel_id in zip(channel_locations, channel_ids)}
print(channel_locations_mappings)



In [ ]:
#thinking the unique locations have all the required electrodes.

required_electrodes= {channels :[int(locations[0]/17.5),int(locations[1]/17.5)] for channels, locations in channel_locations_mappings.items() if channels in unique_channels }
print(required_electrodes)

In [ ]:
import json

with open('electrode_locations_4min.json','w') as fileptr:
    json.dump(required_electrodes,fileptr)



In [ ]:
with open('electrode_locations_4min.json',) as fileptr:
    data = json.load(fileptr)
print(data)

In [ ]:
selected_electrodes = [220 * locations[1] + locations[0] for locations in data.values()]
print(selected_electrodes)

In [ ]:
val = recording_chunk.get_electrode_info
print(val)

In [ ]:
def electrode_rectangle_indices(xmin, ymin, xmax, ymax):
    return [220 * y + x for y in range(max(ymin, 0), min(ymax + 1, 120)) for x in range(max(xmin, 0), min(xmax + 1, 220))]


def electrode_rectangle_um(xmin, ymin, xmax, ymax):
    return electrode_rectangle_indices(int(xmin / 17.5), int(ymin / 17.5), int(xmax / 17.5), int(ymax / 17.5))

XMULTIPLIER = 44
YMULTIPLIER = 20

for y in range(6):
    for x in range(5):
        #print("(",XMULTIPLIER*x,YMULTIPLIER*y,XMULTIPLIER*x+XMULTIPLIER-1,YMULTIPLIER*y+YMULTIPLIER-1,")")
        print(electrode_rectangle_indices(XMULTIPLIER*x,YMULTIPLIER*y,XMULTIPLIER*x+XMULTIPLIER-1,YMULTIPLIER*y+YMULTIPLIER-1))

# print(electrode_rectangle_indices(218,0,219,1))

In [ ]:
import datetime as datetime

now = datetime.datetime.now()
strfmt = now.strftime("%Y%m%d_%H_%M_%S")
print(strfmt)

In [ ]:
import maxlab.saving

obj = maxlab.saving.Saving()

obj.start()

In [ ]:
fig, ax1 = plt.subplots(figsize=(15,5))
spike_times = {}
for idx, unit_id in enumerate(clean_sorting.get_unit_ids()):
    spike_train = clean_sorting.get_unit_spike_train(unit_id,start_frame=1*fs,end_frame=100*fs)
    print(spike_train)
    if len(spike_train) > 0:
        spike_times[idx] = spike_train / float(fs)
        #print(spike_times[unit_id])
       # print(unit_id*np.ones_like(spike_times[unit_id]))
        ax1.plot(spike_times[idx],idx*np.ones_like(spike_times[idx]),
                             marker='|', mew=1, markersize=3,
                             ls='',color='black')
                       

In [ ]:
t_start = 0 
t_end = int(600*fs)
dt = 1
#initialising the spike train.
units= clean_sorting.get_num_units()
frame_numbers = t_end
spike_array = np.zeros((units,frame_numbers), dtype= int)
for idx, unit_id in enumerate(clean_sorting.get_unit_ids()):
    spike_train = clean_sorting.get_unit_spike_train(unit_id,start_frame=t_start,end_frame=t_end)
    for spike_time in spike_train:
        spike_array[idx,spike_time] = 1

print(spike_array)

print(spike_array[0,63782])

In [ ]:
np.savez_compressed('spike_array_compressed_blockactivity.npz',spike_array)

In [ ]:
with np.load('spike_array_compressed_blockactivity.npz') as data:
    decompressed_data = data['arr_0']

print(np.array_equal(spike_array, decompressed_data))


In [ ]:
extremum_channels_ids =spikeinterface.full.get_template_extremum_channel(waveforms, peak_sign='neg')
print(extremum_channels_ids)



In [ ]:
colors = [ 'Lime','Gold', 'Orange','Orangered']
fig, ax = plt.subplots()
wf=[]
for i, unit_id in enumerate([34, 49]):
    wf = waveforms.get_waveforms(unit_id)
    color = colors[i]
    ax.plot(wf[:, :,channel_association_dict[594]].T, color=color, lw=0.3)
print(wf.shape)

In [ ]:
colors = [ 'Fuchsia','Olive', 'Teal']
fig, ax = plt.subplots()
wf=[]
for i, unit_id in enumerate([218]):
    wf = waveform_good.get_waveforms(unit_id)
    ax.plot(wf[:, :,channel_association_dict[902]].T, color=colors[0], lw=0.3)
    ax.plot(wf[:, :,channel_association_dict[613]].T, color=colors[1], lw=0.3)
    ax.plot(wf[:, :,channel_association_dict[663]].T, color=colors[2], lw=0.3)
print(wf.shape)

In [ ]:
peak_shift=si.get_template_extremum_channel_peak_shift(waveform_good)

print(peak_shift)

In [ ]:
colors = [ 'Fuchsia','Olive', 'Teal']
fig, ax = plt.subplots()
for i, unit_id in enumerate([26, 40 , 46]):
    template = waveforms.get_template(unit_id)
    color = colors[i]
    ax.plot(template[:, channel_association_dict[780]].T, color=color, lw=3)
print(template.shape)

In [ ]:
w = sw.plot_unit_templates(waveform_good, unit_ids=[183],plot_channels=False )

In [ ]:
w = sw.plot_unit_waveforms(waveform_good, unit_ids=[2,4,7])
w = sw.plot_unit_templates(waveform_good, unit_ids=[2,4,7])
w = sw.plot_unit_probe_map(waveform_good, unit_ids=[2,4,7])

In [ ]:
    all_units = sortingKS3.get_unit_ids()
    print(f"Total units counts {len(all_units)}")
    print(f"Units : {all_units}")
    rp_contamination,rp_violation = qm.compute_refrac_period_violations(waveforms)
    print(f"rp_contaminaiton : {rp_contamination}")

    refrct_violated_units = [unit for unit,ratio in rp_contamination.items() if ratio >0.0]

    snr_ratio = qm.compute_snrs(waveforms)
    print(f"SNR ratio : {snr_ratio}")
    snr_violated_units = [unit for unit,ratio in snr_ratio.items() if ratio >5]

    firing_rate = qm.compute_firing_rates(waveforms)
    print(f"Firing rate : {firing_rate}")
    fr_violated_units = [unit for unit,ratio in firing_rate.items() if ratio <=0.1]

    violated_units = list(set(refrct_violated_units+fr_violated_units))
    print(f"violated units {violated_units} removed")

    
    good_units = [units for units in all_units if units not in violated_units ]